In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import imutils
import tensorflow as tf
from tensorflow import keras
from typing import Optional, List
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
from DataClasses import Exercise, FrameData, VideoData, Dataset, build_tf_dataset
from Models import ClassifierModel, normalize_label
from pose_detection import extract_pose_from_video_interpolated

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [3]:
num_classes = 22

clf = ClassifierModel(n_classes=num_classes)
model = clf.build_model()
model.summary()

Model: "pointnet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 33, 3)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 33, 32)    │        128 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 33, 32)    │        128 │ conv1d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 33, 32)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 33, 64)    │      2,112 │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 33, 64)    │        256 │ conv1d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 33, 64)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 33, 512)   │     33,280 │ activation_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 33, 512)   │      2,048 │ conv1d_2[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 33, 512)   │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 512)       │          0 │ activation_2[0][… │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │    131,328 │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256)       │      1,024 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 256)       │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │     32,896 │ activation_3[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dense_1[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 128)       │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 9)         │      1,161 │ activation_4[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 3, 3)      │          0 │ dense_2[0][0]   

 Total params: 750,527 (2.86 MB)

 Trainable params: 744,447 (2.84 MB)

 Non-trainable params: 6,080 (23.75 KB)

Model: "pointnet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 33, 3)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 33, 32)    │        128 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 33, 32)    │        128 │ conv1d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 33, 32)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 33, 64)    │      2,112 │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 33, 64)    │        256 │ conv1d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 33, 64)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 33, 512)   │     33,280 │ activation_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 33, 512)   │      2,048 │ conv1d_2[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 33, 512)   │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 512)       │          0 │ activation_2[0][… │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │    131,328 │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256)       │      1,024 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 256)       │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │     32,896 │ activation_3[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dense_1[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 128)       │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 9)         │      1,161 │ activation_4[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 3, 3)      │          0 │ dense_2[0][0]   

 Total params: 750,527 (2.86 MB)

 Trainable params: 744,447 (2.84 MB)

 Non-trainable params: 6,080 (23.75 KB)

In [8]:
# Conversion en dataset de frames
dataset = Dataset()

dataset_raw = pd.read_csv("data/full_landmarks_dataset.csv")  # Pour charger le dataset complet


# Ajout d'une barre de chargement avec tqdm
for index, data in tqdm(dataset_raw.iterrows(), total=len(dataset_raw), desc="Chargement du dataset"):
    frame_data = FrameData(
        filename=data["video_name"], 
        frame_index=data["frame_number"],
        landmarks=np.array(data.drop(["video_name","total_frames","frame_number","width","height","label"])).reshape(-1, 3),
        predicted_class=None,
        confidence=None,
        scores=None,
        ground_truth=Exercise(normalize_label(data["label"]))
    )
    dataset.add_data(frame_data)

print(f"Dataset chargé avec succès ! Total: {len(dataset.datas)} frames")

C:\Users\Arthur\AppData\Local\Temp\ipykernel_15632\1209185612.py:4: DtypeWarning: Columns (104) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset_raw = pd.read_csv("data/full_landmarks_dataset.csv")  # Pour charger le dataset complet
Chargement du dataset: 100%|██████████| 103890/103890 [01:47<00:00, 968.50it/s] 

Dataset chargé avec succès ! Total: 103890 frames


In [10]:
# Split train / test
print("📊 Division du dataset...")
train_dataset, test_dataset = dataset.split(train_ratio=0.8)

# Vérification des classes dans chaque split
train_X, train_y = train_dataset.get_data_arrays()
test_X, test_y = test_dataset.get_data_arrays()

print(f"Classes d'entraînement uniques: {np.unique(train_y)}")
print(f"Classes de test uniques: {np.unique(test_y)}")

# === 3️⃣ Construction des datasets TensorFlow ===
print("🔧 Construction des datasets TensorFlow...")
train_tf_dataset = build_tf_dataset(train_dataset, num_classes=num_classes, batch_size=32)
test_tf_dataset = build_tf_dataset(test_dataset, num_classes=num_classes, batch_size=32)

# Vérification des formes
for batch in train_tf_dataset.take(1):
    X_batch, y_batch = batch
    print(f"Forme des batches d'entraînement: X={X_batch.shape}, y={y_batch.shape}")

# === 4️⃣ Entraînement ===
print("🎯 Début de l'entraînement du modèle...")
history = clf.train_model(train_tf_dataset, test_tf_dataset, epochs=3, lr=0.001)

# Affichage des courbes d'entraînement
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

📊 Division du dataset...
Split completed:
  Training: 82665 frames from 361 videos
  Testing: 21225 frames from 91 videos


ValueError: could not convert string to float: '0.6691516637802124barbell'

In [6]:
# Exemple avec 2 vidéos pour tester
video1 = VideoData(filename="data/test_bench_press.mp4", ground_truth=Exercise.BENCH_PRESS)
video1.landmark_estimation()
video1.normalize()

dataset = Dataset()
dataset.add_video_data(video1)


🔍 Processing video frames...


Reading frames: 100%|███████████████████████████| 53/53 [00:13<00:00,  4.05it/s]


✅ Interpolation done.
💾 Landmarks saved to interpolated_landmarks.txt
▶ Replaying with interpolated landmarks...


Displaying: 100%|███████████████████████████████| 53/53 [00:02<00:00, 26.31it/s]


===== End of visualization =====


In [7]:
for frame_data in dataset.datas:
    clf.predict_frame(frame_data)
    print(f"Vidéo: {frame_data.filename}, Frame: {frame_data.frame_index}, Prédit: {frame_data.predicted_class}, Vérité: {frame_data.ground_truth}")

Vidéo: data/test_bench_press.mp4, Frame: 0, Prédit: Exercise.TRICEP_PUSHDOWN, Vérité: Exercise.BENCH_PRESS
Vidéo: data/test_bench_press.mp4, Frame: 1, Prédit: Exercise.TRICEP_PUSHDOWN, Vérité: Exercise.BENCH_PRESS
Vidéo: data/test_bench_press.mp4, Frame: 2, Prédit: Exercise.TRICEP_PUSHDOWN, Vérité: Exercise.BENCH_PRESS
Vidéo: data/test_bench_press.mp4, Frame: 3, Prédit: Exercise.TRICEP_PUSHDOWN, Vérité: Exercise.BENCH_PRESS
Vidéo: data/test_bench_press.mp4, Frame: 4, Prédit: Exercise.TRICEP_PUSHDOWN, Vérité: Exercise.BENCH_PRESS
Vidéo: data/test_bench_press.mp4, Frame: 5, Prédit: Exercise.TRICEP_PUSHDOWN, Vérité: Exercise.BENCH_PRESS
Vidéo: data/test_bench_press.mp4, Frame: 6, Prédit: Exercise.TRICEP_PUSHDOWN, Vérité: Exercise.BENCH_PRESS
Vidéo: data/test_bench_press.mp4, Frame: 7, Prédit: Exercise.TRICEP_PUSHDOWN, Vérité: Exercise.BENCH_PRESS
Vidéo: data/test_bench_press.mp4, Frame: 8, Prédit: Exercise.TRICEP_PUSHDOWN, Vérité: Exercise.BENCH_PRESS
Vidéo: data/test_bench_press.mp4, Fra